In [48]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn.preprocessing import LabelBinarizer
import scipy.stats as stats
from sklearn import linear_model
from scipy.stats import ttest_rel

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

In [2]:
january_flight_data = pd.read_csv("January Flight Data.csv")
january_flight_data_df= january_flight_data[['Flight Date','Flight Day', 'Flight Month', 'Flight Year', 'Airline','ORIGIN','Destination','Departure Time','CRS Departure Time',
        'Taxi Out', 'Departure Delay', 'Air Time', 'Distance','Cancelled', 'Tail Number','ICAO Type','Manufacture Year','Temperature',
        'Cloud Cover','Active Weather']]

In [ ]:
january_flight_data_df.head()

In [ ]:
january_flight_data.describe()

In [3]:
y = LabelBinarizer().fit_transform(january_flight_data_df["Active Weather"])

In [4]:
active_weather = pd.DataFrame(y,columns = ["No Weather","Moderate Weather", "Severe Weather"])

In [5]:
df = pd.concat([january_flight_data_df, active_weather],axis = 1, join = "inner")

In [6]:
df.head()

,Flight Date,Flight Day,Flight Month,Flight Year,Airline,ORIGIN,Destination,Departure Time,CRS Departure Time,Taxi Out,...,Cancelled,Tail Number,ICAO Type,Manufacture Year,Temperature,Cloud Cover,Active Weather,No Weather,Moderate Weather,Severe Weather
0,2022-01-01,1,1,2022,American Airlines,JFK,LAX,2022-01-01 06:57:00,2022-01-01 07:00:00,19,...,0,N106NN,A321,2014,9.39,4.0,0.0,1,0,0
1,2022-01-01,1,1,2022,American Airlines,LAX,JFK,2022-01-01 08:57:00,2022-01-01 09:00:00,20,...,0,N101NN,A321,2013,12.22,2.0,0.0,1,0,0
2,2022-01-01,1,1,2022,American Airlines,ORD,FLL,2022-01-01 09:37:00,2022-01-01 18:30:00,23,...,0,N805NN,B738,2009,2.22,4.0,1.0,0,1,0
3,2022-01-01,1,1,2022,American Airlines,CLT,STL,2022-01-01 23:00:00,2022-01-01 22:40:00,16,...,0,N870NN,B738,2011,20.00,3.0,0.0,1,0,0
4,2022-01-01,1,1,2022,American Airlines,ATL,DFW,2022-01-01 16:24:00,2022-01-01 16:29:00,30,...,0,N561UW,A321,2012,21.72,4.0,1.0,0,1,0


# Explanation

With our data frame create, we run a groupby, grouping by Airports, so that we can see the how each airport's details. We need to then build out the measures to view and combine them into a dataframe to run our tests.

In [9]:
origin_caused_delay = df.groupby("ORIGIN")

In [11]:
origin_delay = origin_caused_delay['ORIGIN','Air Time','Distance','Departure Delay']

c:\Users\evan-\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [12]:
origin_delay = origin_delay.mean()

In [19]:
no_active_weather = df[(df['No Weather'] == 1)].groupby('ORIGIN')['No Weather'].sum()

In [20]:
moderate_active_weather = df[(df['Moderate Weather'] == 1)].groupby('ORIGIN')['Moderate Weather'].sum()

In [21]:
severe_active_weather = df[(df['Severe Weather'] == 1)].groupby('ORIGIN')['Severe Weather'].sum()

In [22]:
flight_count = origin_caused_delay['ORIGIN'].count()

In [23]:
origin_delay['Total Flights in January'] = flight_count

In [27]:
origin_delay["Flights per Day"] = origin_delay["Total Flights in January"]/31

In [17]:
origin_delay['No Active Weather'] = no_active_weather

In [24]:
origin_delay['Moderate Weather'] = moderate_active_weather

In [26]:
origin_delay['Severe Weather'] = severe_active_weather

In [43]:
origin_delay["Total Taxi Out Time"] = origin_caused_delay["Taxi Out"].sum()
origin_delay["Taxi Time Per Flight"] = origin_delay["Total Taxi Out Time"] / origin_delay["Total Flights in January"] 

In [44]:
origin_delays = origin_delay.fillna(0).sort_values(by = "Departure Delay", ascending = False)

In [45]:
origin_delays

,Air Time,Distance,Departure Delay,No Active Weather,Total Flights in January,Moderate Weather,Severe Weather,Flights per Day,Total Taxi Out Time,Taxi Time Per Flight
ORIGIN,,,,,,,,,,
EYW,108.944444,823.380051,30.441919,768,792,23.0,1.0,25.548387,10755,13.579545
JAC,101.800388,796.160853,28.740310,360,516,153.0,3.0,16.645161,12145,23.536822
STT,184.649194,1369.695565,26.129032,492,496,4.0,0.0,16.000000,7238,14.592742
DEN,105.947547,851.465588,18.921105,17708,20952,2863.0,381.0,675.870968,378472,18.063765
ROC,63.766384,398.164996,17.696993,700,1297,591.0,6.0,41.838710,26274,20.257517
...,...,...,...,...,...,...,...,...,...,...
ILM,63.029979,385.706638,4.224839,429,467,30.0,8.0,15.064516,6840,14.646681
SNA,102.131652,774.191242,4.065965,3245,3608,363.0,0.0,116.387097,45266,12.546009
SAN,131.272547,1061.605221,4.050945,5099,5555,456.0,0.0,179.193548,82334,14.821602


With our dataframe built, we can run describe() to get a general desrciption of our columns.

In [49]:
origin_delays.describe()

,Air Time,Distance,Departure Delay,No Active Weather,Total Flights in January,Moderate Weather,Severe Weather,Flights per Day,Total Taxi Out Time,Taxi Time Per Flight
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,108.746662,807.424977,10.612840,5262.402778,6063.930556,776.125000,25.402778,195.610663,103992.625000,15.879680
std,28.769751,251.606959,5.228199,5323.012492,5999.190916,1046.285205,58.126782,193.522288,112794.528466,3.021507
min,63.029979,385.706638,3.168251,103.000000,108.000000,0.000000,0.000000,3.483871,1405.000000,11.295366
25%,90.989200,641.094019,7.492193,1512.250000,1589.750000,126.250000,0.000000,51.282258,24210.250000,13.501429
50%,102.107903,750.153824,9.745825,3075.000000,3539.500000,368.000000,6.000000,114.177419,54022.000000,15.159447
75%,122.183574,947.602315,12.359226,7419.250000,9730.250000,975.250000,21.000000,313.879032,173819.750000,17.855430
max,188.898191,1574.472932,30.441919,22349.000000,25277.000000,5886.000000,381.000000,815.387097,497040.000000,23.536822


Now we can run our tests. We first run a Multi-Linear regression so we can figure out which column is most influential in causing departure delays and after that, we can verify that it affects the dependent variable by running a t test and confirming it against the Null Hypothesis of zero affect.

# Multi-Linear Regression

In [47]:
X = origin_delays[['Air Time', 'Distance', 'No Active Weather', 'Moderate Weather', 'Severe Weather', 'Flights per Day', 'Taxi Time Per Flight']]
y = origin_delays['Departure Delay']

regr = linear_model.LinearRegression()
regr.fit(X, y)

print(regr.coef_)

[ 6.32974996e-02  1.34188041e-04 -3.25393955e-04 -7.45869915e-04
  2.04560327e-02  6.25315125e-04  9.61909419e-01]


A Multi-Linear works by building out an equation for a line, much like the single linear regression but it uses multiple slopes to find the variable that impacts the y variable (Departure Delays in this case) the most. In our case, each variable seems to impact Departure Delays only a slight amount so that would lead us to think that the main cause of flight delays are something else entirely outside of our dataset. But in this case, the Taxi Time Per Flight seems to be the main cause. So we can take Taxi Flight Time and use it in our hypotheisis.

# Null Hypothesis
Taxi Flight Times have no affect on Flight Delays

# t-Test

In [58]:
#group1 = origin_delays[origin_delays['Departure Delay']]
#group2 = origin_delays[origin_delays['Taxi Time Per Flight']]
stats.ttest_ind(origin_delays['Taxi Time Per Flight'], origin_delays['Departure Delay'],equal_var = False)

Ttest_indResult(statistic=7.400937226766764, pvalue=2.5218024808967977e-11)

# Conclusion
With a p-value of less than .05, we fail to reject the null hypothesis, meaning that Taxi Time Per Flight has an affect on Departure Delays.